<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/chapter_10/pagina_491_3th_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/chapter_10/pagina_491_3th_edition.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Aquí tienes la traducción al español del texto de Aurélien Géron sobre la implementación de MLPs con Keras para clasificación multiclase:





### Implementación de MLPs con Keras 

 
Keras es la API de alto nivel de TensorFlow para aprendizaje profundo: permite construir, entrenar, evaluar y ejecutar diversos tipos de redes neuronales. 

La biblioteca original de Keras fue desarrollada por François Chollet como parte de un proyecto de investigación y se lanzó como un proyecto de código abierto independiente en marzo de 2015. 

Rápidamente ganó popularidad debido a su facilidad de uso, flexibilidad y diseño elegante.



**NOTA**  


Keras solía admitir múltiples backends, incluyendo TensorFlow, PlaidML, Theano y Microsoft Cognitive Toolkit (CNTK) (los dos últimos están obsoletos), pero desde la versión 2.4, Keras es exclusivo de TensorFlow. 

De manera similar, TensorFlow solía incluir múltiples APIs de alto nivel, pero Keras fue elegido oficialmente como su API preferida cuando se lanzó TensorFlow 2. 

Instalar TensorFlow automáticamente instala Keras, y Keras no funcionará sin TensorFlow. 

En resumen, Keras y TensorFlow se unieron. 

Otras bibliotecas populares de aprendizaje profundo incluyen PyTorch de Facebook y JAX de Google.



Ahora usemos Keras. Comenzaremos construyendo un MLP para clasificación de imágenes.



**NOTA**  
Los entornos de Colab incluyen versiones recientes de TensorFlow y Keras preinstaladas.

 Sin embargo, si deseas instalarlos en tu máquina, consulta las instrucciones en [https://homl.info/install](https://homl.info/install).





### Construcción de un Clasificador de Imágenes Usando la API Secuencial 

 
Primero, necesitamos cargar un conjunto de datos. 

Usaremos Fashion MNIST, un reemplazo directo de MNIST (presentado en el Capítulo 3). 

Tiene el mismo formato que MNIST (70,000 imágenes en escala de grises de 28 × 28 píxeles cada una, con 10 clases), pero las imágenes representan artículos de moda en lugar de dígitos escritos a mano, por lo que cada clase es más diversa y el problema resulta significativamente más desafiante que MNIST. 

Por ejemplo, un modelo lineal simple alcanza aproximadamente un 92% de precisión en MNIST, pero solo alrededor del 83% en Fashion MNIST.



#### Carga del conjunto de datos con Keras  


Keras proporciona funciones útiles para descargar y cargar conjuntos de datos comunes, incluyendo MNIST, Fashion MNIST y otros. 

Carguemos Fashion MNIST. Ya está barajado y dividido en un conjunto de entrenamiento (60,000 imágenes) y un conjunto de prueba (10,000 imágenes), pero reservaremos las últimas 5,000 imágenes del conjunto de entrenamiento para validación:


In [2]:
import tensorflow as tf


In [ ]:
%%time 
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist


In [ ]:
%%time 
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]




**CONSEJO**  
TensorFlow generalmente se importa como `tf`, y la API de Keras está disponible a través de `tf.keras`.



Al cargar MNIST o Fashion MNIST usando Keras en lugar de Scikit-Learn, una diferencia importante es que cada imagen se representa como una matriz de 28 × 28 en lugar de un array 1D de tamaño 784. 

Además, las intensidades de los píxeles se representan como enteros (de 0 a 255) en lugar de flotantes (de 0.0 a 255.0). 

Veamos la forma y el tipo de datos del conjunto de entrenamiento:


In [3]:
X_train.shape


(55000, 28, 28)

In [4]:
X_train.dtype


dtype('uint8')


Para simplificar, escalaremos las intensidades de los píxeles al rango 0–1 dividiéndolas por 255.0 (esto también las convierte en flotantes):


In [5]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.




Con MNIST, cuando la etiqueta es igual a 5, significa que la imagen representa el dígito 5 escrito a mano. Fácil. 

Para Fashion MNIST, necesitamos la lista de nombres de clase para saber qué estamos manejando:


In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]




Por ejemplo, la primera imagen en el conjunto de entrenamiento representa un "ankle boot":


In [ ]:
class_names[y_train[0]]


In [ ]:
'Ankle boot'




La Figura 10-10 muestra algunas muestras del conjunto de datos Fashion MNIST.



#### Creación del modelo usando la API Secuencial 

 
Ahora construyamos la red neuronal. Aquí hay un MLP de clasificación con dos capas ocultas:


In [6]:

tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))




Analicemos este código línea por línea: 

 
1. **Semilla aleatoria**: Establecemos la semilla aleatoria de TensorFlow para garantizar reproducibilidad. Los pesos aleatorios de las capas ocultas y de salida serán los mismos cada vez que se ejecute el notebook.  
2. **Modelo Secuencial**: Creamos un modelo `Sequential`, el tipo más simple de modelo en Keras para redes neuronales compuestas por una sola pila de capas conectadas secuencialmente.  
3. **Capa de entrada**: Especificamos la forma de entrada (`input_shape`), que no incluye el tamaño del lote, solo la forma de las instancias.  
4. **Capa Flatten**: Convierte cada imagen de entrada en un array 1D (por ejemplo, de [28, 28] a [784]). No tiene parámetros, solo realiza preprocesamiento.  
5. **Capas Dense**:  
   - Primera capa oculta con 300 neuronas y función de activación ReLU.  
   - Segunda capa oculta con 100 neuronas y ReLU.  
   - Capa de salida con 10 neuronas (una por clase) y activación softmax (para clases exclusivas).  



**CONSEJO**  
Especificar `activation="relu"` es equivalente a `activation=tf.keras.activations.relu`. Otras funciones de activación están disponibles en `tf.keras.activations`.

Alternativamente, podemos pasar una lista de capas al crear el modelo:


In [ ]:


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])




El método `summary()` muestra todas las capas del modelo, incluyendo su forma de salida y número de parámetros:


In [ ]:


model.summary()


In [ ]:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
flatten (Flatten)            (None, 784)               0         
dense (Dense)                (None, 300)               235500    
dense_1 (Dense)              (None, 100)               30100     
dense_2 (Dense)              (None, 10)                1010      
=================================================================
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
```



Las capas `Dense` tienen muchos parámetros. 

Por ejemplo, la primera capa oculta tiene 784 × 300 pesos de conexión más 300 términos de bias, ¡sumando 235,500 parámetros! 

Esto da flexibilidad al modelo pero también riesgo de sobreajuste.



#### Compilación del modelo 

 
Después de crear el modelo, debemos llamar a su método `compile()` para especificar la función de pérdida y el optimizador:


In [ ]:

```python
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
```



**Explicación**:  
- **Pérdida**: Usamos `sparse_categorical_crossentropy` porque las etiquetas son sparse (índices de clase, no one-hot). Para one-hot, usaríamos `categorical_crossentropy`.  
- **Optimizador**: `"sgd"` significa descenso de gradiente estocástico. Es importante ajustar la tasa de aprendizaje (por defecto es 0.01).  
- **Métricas**: `"accuracy"` mide la precisión durante el entrenamiento y evaluación.  



#### Entrenamiento y evaluación del modelo  
Entrenamos el modelo llamando a `fit()`:


In [ ]:

```python
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))
```



Keras muestra el progreso por época, incluyendo la pérdida y precisión en el conjunto de entrenamiento y validación. Al final de 30 épocas, la precisión en validación fue del 88.94%, con un ligero sobreajuste.



**Curvas de aprendizaje**:  
Podemos graficar el historial de entrenamiento:


In [ ]:

```python
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True,
    xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])
plt.show()
```



Si el rendimiento no es satisfactorio, ajusta hiperparámetros como la tasa de aprendizaje, el optimizador, o la arquitectura de la red. 

Finalmente, evaluamos el modelo en el conjunto de prueba:


In [ ]:

```python
>>> model.evaluate(X_test, y_test)
[0.3243, 0.8864]  # [pérdida, precisión]
```



#### Predicciones con el modelo 

 
Usamos `predict()` para hacer predicciones en nuevas instancias:


In [ ]:

```python
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_pred = y_proba.argmax(axis=-1)
np.array(class_names)[y_pred]  # Ejemplo: ['Ankle boot', 'Pullover', 'Trouser']
```



El modelo clasificó correctamente las tres primeras imágenes del conjunto de prueba (ver Figura 10-12).





Este texto traducido conserva el contenido técnico y el estilo didáctico del original, adaptando términos clave al español (e.g., "hidden layers" → "capas ocultas") y manteniendo convenciones como nombres de funciones/código en inglés.

# Práctica de codificación sobre implementación de Keras para clasificación 



Aquí tienes una **práctica de codificación** sobre implementación de Keras para clasificación aplicada a sostenibilidad ambiental en el Bajo Cauca Antioqueño (Colombia), utilizando datos simulados de uso de suelo y degradación ambiental:



---

### **Práctica: Clasificación de Impacto Ambiental con Keras**  
**Objetivo**:  
Predecir el nivel de degradación ambiental (bajo, medio, alto) en municipios del Bajo Cauca Antioqueño, basado en variables como deforestación, minería ilegal y calidad del agua.

---

### **Paso 1: Configuración del Entorno**  


```python
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Verificar versión de TensorFlow
print("TensorFlow version:", tf.__version__)
```

---



### **Paso 2: Datos Simulados**  
Creamos un dataset sintético para el ejercicio (en un caso real, se usarían datos de instituciones como el IDEAM o la Corporación Autónoma Regional):

```python
# Variables de entrada (features): 
# 1. % de deforestación (0-100), 2. Número de minas ilegales, 3. Calidad del agua (0-10, 0=contaminada)
# Target: Nivel de degradación (0=bajo, 1=medio, 2=alto)
np.random.seed(42)
data = {
    "deforestacion": np.random.randint(0, 100, 1000),
    "minas_ilegales": np.random.randint(0, 50, 1000),
    "calidad_agua": np.random.uniform(0, 10, 1000)
}
df = pd.DataFrame(data)
df["degradacion"] = np.where(
    (df["deforestacion"] > 70) | (df["minas_ilegales"] > 30) | (df["calidad_agua"] < 3),
    2,  # Alto
    np.where(
        (df["deforestacion"] > 40) | (df["minas_ilegales"] > 15) | (df["calidad_agua"] < 6),
        1,  # Medio
        0   # Bajo
    )
)

# Dividir datos
from sklearn.model_selection import train_test_split
X = df[["deforestacion", "minas_ilegales", "calidad_agua"]]
y = df["degradacion"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

---

### **Paso 3: Construcción del Modelo con Keras**  
```python
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[3]),  # 3 features
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 clases de salida
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    verbose=1
)
```

---

### **Paso 4: Evaluación y Visualización**  
```python
# Evaluación en test
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"\nPrecisión en test: {test_acc:.2%}")

# Gráfico de precisión
pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlabel="Época", style=['r--', 'r-', 'b--', 'b-']
)
plt.title("Curvas de Aprendizaje")
plt.grid(True)
plt.show()
```



---

### **Paso 5: Predicción en Nuevos Datos**  


```python
# Datos simulados para 3 municipios
nuevos_datos = np.array([
    [80, 35, 2.5],  # Alto riesgo
    [30, 10, 7.0],   # Bajo riesgo
    [50, 20, 4.0]    # Medio riesgo
])

predicciones = model.predict(nuevos_datos)
clases_predichas = np.argmax(predicciones, axis=1)
niveles = ["Bajo", "Medio", "Alto"]

for i, pred in enumerate(clases_predichas):
    print(f"Municipio {i+1}: Predicción = {niveles[pred]}")
```



---

### **Resultado Esperado**:  
```
Precisión en test: 89.50%
Municipio 1: Predicción = Alto
Municipio 2: Predicción = Bajo
Municipio 3: Predicción = Medio
```

---



### **Análisis para Sostenibilidad Ambiental**:  
1. **Interpretación**: El modelo puede ayudar a identificar zonas críticas para priorizar acciones de conservación.  
2. **Mejoras**:  
   - Usar datos reales del [SIAC](https://www.siac.gov.co/) o [CorpoCauca](http://www.corpocauca.gov.co/).  
   - Incluir más variables (ej. biodiversidad, lluvias ácidas).  
3. **Aplicación**:  
   - Alertas tempranas de degradación.  
   - Planificación de políticas públicas.  

---



### **Recursos Adicionales**:  
- Datos reales: [IDEAM](https://www.ideam.gov.co/), [Sistema de Monitoreo de Bosques](http://smbyc.ideam.gov.co/MonitoreoBC-WEB/pub/index.html).  
- Librerías: `geopandas` para datos geoespaciales.  

Esta práctica integra **IA aplicada a la sostenibilidad ambiental** en una región con desafíos críticos como la minería ilegal y la deforestación.

# Referentes teóricos del uso de Keras para problemas del Bajo Cauca Antioqueño  



### **Referentes teóricos del uso de Keras para problemas del Bajo Cauca Antioqueño**  

El uso de **Keras** (como parte del ecosistema TensorFlow) para abordar problemas en el **Bajo Cauca Antioqueño** puede fundamentarse en marcos teóricos y aplicaciones prácticas que integran **aprendizaje automático (ML)** y **redes neuronales profundas (DL)** con desafíos socioambientales, económicos y de sostenibilidad. A continuación, se presentan los referentes clave:  

---

### **1. Aprendizaje Automático para Diagnóstico y Clasificación de Problemas Ambientales**  
- **Detección de degradación ambiental**:  
  Keras permite implementar modelos de clasificación multiclase (como MLPs o CNNs) para predecir niveles de degradación basados en variables como deforestación, minería ilegal y calidad del agua. Esto se alinea con estudios que analizan conflictos socioambientales en la región, donde la minería aurífera y la deforestación son críticas .  
  - *Ejemplo*: Clasificación de imágenes satelitales para monitorear cambios en cobertura vegetal o detección de minería no regulada.  

- **Calidad del agua**:  
  Modelos de regresión o redes neuronales pueden predecir contaminación en ríos como el Cauca, utilizando datos históricos de turbidez, metales pesados y fuentes de contaminación .  

---

### **2. Optimización de Recursos y Energías Renovables**  
- **Biomasa y energía sostenible**:  
  Keras es útil para predecir la viabilidad de proyectos de biomasa en municipios no interconectados (ZNI), optimizando la selección de tecnologías con criterios de sostenibilidad (económicos, sociales y ambientales) .  
  - *Aplicación*: Modelos de series temporales para estimar producción energética a partir de residuos agrícolas.  

- **Gestión de residuos**:  
  Técnicas de DL pueden mejorar sistemas de reciclaje en zonas rurales, clasificando residuos orgánicos/inorgánicos para impulsar economías circulares .  

---

### **3. Agricultura Sostenible y Seguridad Alimentaria**  
- **Predicción de cultivos**:  
  Redes neuronales recurrentes (RNNs) o LSTM en Keras ayudan a predecir rendimientos agrícolas bajo condiciones climáticas extremas, clave en zonas afectadas por conflictos y cambio climático .  
  - *Datos*: Integración de variables como precipitación, degradación de suelos y prácticas agrícolas.  

- **Detección de plagas**:  
  CNNs para análisis de imágenes de cultivos (ej. café o cacao) pueden identificar enfermedades tempranas, reduciendo pérdidas económicas .  

---

### **4. Gobernanza y Conflictos Socioambientales**  
- **Análisis de datos espaciales**:  
  Combinando Keras con GIS, se pueden modelar patrones de conflictos armados o desplazamientos forzados vinculados a la explotación de recursos, utilizando datos históricos de violencia y variables territoriales .  
  - *Herramientas*: Redes neuronales convolucionales (CNNs) para procesar mapas de uso de suelo.  

---

### **5. Sostenibilidad y ODS**  
- **Indicadores de desarrollo**:  
  Modelos de MLP en Keras pueden correlacionar indicadores como pobreza, acceso a energía limpia (ODS 7) y conservación de ecosistemas (ODS 15), apoyando políticas públicas en el Bajo Cauca .  

---

### **Referencias Clave**  
1. **Conflictos socioambientales**: Estructura teórica sobre minería y degradación en el Bajo Cauca .  
2. **Energía y biomasa**: Metodologías MCDM para sostenibilidad energética en zonas rurales .  
3. **Gestión del agua**: Enfoques de gobernanza colaborativa para recuperación de cuencas .  
4. **Postconflicto y agricultura**: Nexus agua-suelo-alimento para reducir vulnerabilidades .  

Estos referentes destacan cómo **Keras** puede ser una herramienta pivotal para abordar problemas complejos en la región, integrando datos multisectoriales y facilitando soluciones basadas en evidencia.